# Predicting Success: Machine Learning Models for CBT Outcomes

In [4]:
from google.colab import drive
drive.mount("/content/drive")

import sys
file_path = '/content/drive/MyDrive/Colab Notebooks/'
sys.path.append(file_path)

import cleaning_functions as cleanf
import eda_functions as edaf
import preparation_functions as prepf
import modelling_functions as modelf

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!cat '/content/drive/MyDrive/Colab Notebooks/DissFunctions.py'

{"nbformat":4,"nbformat_minor":0,"metadata":{"colab":{"provenance":[],"authorship_tag":"ABX9TyPErySvhR6MH3vMn0fXcqQw"},"kernelspec":{"name":"python3","display_name":"Python 3"},"language_info":{"name":"python"}},"cells":[{"cell_type":"markdown","metadata":{"id":"W1ODmQunSVNP"},"source":["# Functions"]},{"cell_type":"markdown","metadata":{"id":"QV1f10TTIv53"},"source":["## 1.2 Preprocessing Functions"]},{"cell_type":"code","execution_count":null,"metadata":{"id":"8vw4-ITlmfz1"},"outputs":[],"source":["def load_data():\n","\n","    drive.mount(\"/content/drive\") # access google drive folder\n","\n","    file_path = '/content/drive/MyDrive/Data/Dissertation_Data/'\n","    df = pd.read_csv(file_path + 'mental_health.csv', delimiter = ',') # csv\n","    raw_df = df.copy()\n","\n","    return df, raw_df"]},{"cell_type":"code","execution_count":null,"metadata":{"id":"2nLysn57q38W"},"outputs":[],"source":["def manage_df(df, name, option):\n","\n","    diss_path = '/content/drive/MyDrive/Data/

# A1) Introduction

This study aims to utilise machine learning techniques to predict the likelihood of success in Cognitive Behavioral Therapy (CBT) using data. The dataset encompasses various demographic, treatment-related, and psychological assessment variables, offering insights into patient profiles and treatment outcomes.  By leveraging various analytical skills, predictive models will aid in revealing the success probability for CBT in individual patients.

## 1.2 Preprocessing Functions

In [ ]:
def load_data():

    drive.mount("/content/drive") # access google drive folder

    file_path = '/content/drive/MyDrive/Data/Dissertation_Data/'
    df = pd.read_csv(file_path + 'mental_health.csv', delimiter = ',') # csv
    raw_df = df.copy()

    return df, raw_df

In [ ]:
def manage_df(df, name, option):

    diss_path = '/content/drive/MyDrive/Data/Dissertation_Data/'

    if option == 'save':
        df.to_csv(file_path + name + '.csv')
        print('Saved!')

    elif option == 'load':
        df = pd.read_csv(file_path + name + '.csv', index_col=0)
        return df

2.2 Non-Questionnaire Data Preprocessing

In [ ]:
def rename_features(df): # non-questionnaire features

    df.rename(columns = {
        'Unnamed: 0': 'CaseID',
        'IAPTus_Num': 'ClientID',
        'Referral Date': 'ReferralDate',
        'Age_ReferralRequest_ReceivedDate': 'AgeAtReferralRequest',
        'EthnicDescGroupCode': 'EthnicCode',
        'EthnicCategoryGroupShortCode': 'EthnicCodeShort',
        'GenderIdentity': 'Gender',
        'SexualOrientationDesc': 'SexualOrientation',
        'EndDescGroupShort': 'Treated',
        'AllocatedCareProfNameCode': 'TherapistID',
        'JobTitleCode': 'ExperienceLevel',
        'Days to first assessment': 'DaystoAssessment',
        'Days to first treatment': 'DaystoTreatment',
        'CountOfAttendedCareContacts': 'CareContacts',
        'RecoveryDesc': 'Recovery',
        'ReliableRecoveryDesc': 'ReliableRecovery',
        'Date': 'DateOfQuestionnaire'},
        inplace = True)

    return df

In [ ]:
def create_referral_count(df):
    def count_referrals(col):
        if '_1' in col:
            return 1
        elif '_2' in col:
            return 2
        elif '_3' in col:
            return 3
        elif '_4' in col:
            return 4
        elif '_5' in col:
            return 5
        else:
            return 1
    df.insert(2, "ReferralCount", df['ClientID'].apply(count_referrals)) # introduce next to ClientID
    return df

def clean_client_id(df):
    for text in ['_1', '_2', '_3', '_4']:
        df['ClientID'] = df['ClientID'].str.replace(text, '') # remove ending
    df['ClientID'] = pd.to_numeric(df['ClientID'])
    return df

def convert_features_to_datetime(df):
    df['ReferralDate'] = pd.to_datetime(df['ReferralDate'], format = '%d/%m/%Y')
    df['DateOfQuestionnaire'] = pd.to_datetime(df['DateOfQuestionnaire'], format = '%d/%m/%Y')
    return df

def convert_float_features_to_int(df):
    df['EthnicCode'] = df['EthnicCode'].astype('Int64') # Int deals with NaNs, int does not
    df['EthnicCodeShort'] = df['EthnicCodeShort'].astype('Int64')
    df['TherapistID'] = df['TherapistID'].astype('Int64')
    df['ExperienceLevel'] = df['ExperienceLevel'].astype('Int64')
    return df

In [ ]:
def map_features(df):

    Gender_map = {
        'CHANGE ME': np.nan,
        'X': np.nan}
    df['Gender'] = df['Gender'].replace(Gender_map).astype('Int64')

    Treated_map = {
        'Seen and treated': 1,
        'Seen but not treated': 0}
    df['Treated'] = df['Treated'].replace(Treated_map).astype('Int64')

    ReliableChangeDesc_map = {
        'Reliable improvement': 2, # what about(-1, 0, 1)?
        'No reliable change': 1,
        'Reliable deterioration': 0,
        'Not applicable': np.nan}
    df['ReliableChangeDesc'] = df['ReliableChangeDesc'].replace(ReliableChangeDesc_map).astype('Int64')

    Recovery_map = {
        'At recovery': 1,
        'Not at recovery': 0,
        'Not applicable': np.nan}
    df['Recovery'] = df['Recovery'].replace(Recovery_map).astype('Int64')

    ReliableRecovery_map = {
        'Reliable recovery': 1,
        'No reliable recovery': 0,
        'Not applicable': np.nan}
    df['ReliableRecovery'] = df['ReliableRecovery'].replace(ReliableRecovery_map).astype('Int64')

    return df


In [ ]:
def one_hot_encode_features(df):

    EndDesc_cols = pd.get_dummies(df['EndDesc'], prefix = 'EndDesc')
    EndDesc_index = df.columns.get_loc('EndDesc')
    df = pd.concat([df.iloc[:, :EndDesc_index + 1], EndDesc_cols, df.iloc[:, EndDesc_index + 1:]], axis = 1)
    df = df.drop(columns = ['EndDesc'])

    EndDescShort_cols = pd.get_dummies(df['EndDescShort'], prefix = 'EndDescShort')
    EndDescShort_index = df.columns.get_loc('EndDescShort')
    df = pd.concat([df.iloc[:, :EndDescShort_index + 1], EndDescShort_cols, df.iloc[:, EndDescShort_index + 1:]], axis = 1)
    df = df.drop(columns = ['EndDescShort'])

    return df

def convert_to_int_features(df):
    int_cols = ['SexualOrientation', 'DaystoAssessment', 'DaystoTreatment', 'CareContacts']
    for col in int_cols:
        df[col] = df[col].astype('Int64')
    return df

In [ ]:
def plot_features(df):

    plot_cols = 4
    plot_rows = len(df.columns)//4 + 1

    plt.figure(figsize = (plot_cols*3, plot_rows*3))

    for i, col in enumerate(df.columns, start = 1):
        if df[col].dtype in ['int64', 'float64', 'Int64', 'datetime64[ns]', 'bool']:
            data = df[col].dropna()
            if df[col].dtype == 'bool':  # convert boolean to integers
                data = data.astype(int)
            plt.subplot(plot_rows, 4, i)
            plt.hist(data, bins = 20, color='darkgrey', edgecolor='white')
            plt.title(col)
            plt.xlabel(col)
            plt.ylabel('Frequency')

    plt.suptitle('Data Plots', y = 1, fontsize = 24)
    plt.tight_layout()
    plt.show()

In [ ]:
def preprocess_nonques_features(df):

    df = rename_features(df)
    df = create_referral_count(df)
    df = clean_client_id(df)
    df = convert_features_to_datetime(df)
    df = convert_float_features_to_int(df)
    df = map_features(df)
    df = one_hot_encode_features(df)
    df = convert_to_int_features(df)

    return df

2.3 Questionnaire Data Preprocessing

In [ ]:
def preprocess_ques_features(df):

    # convert all variables to float
    for col in df.columns[27:]:
        df[col] = pd.to_numeric(df[col], errors = 'coerce') # (I think this also removes '.')

    item_cols = df.columns.str.contains('Item').tolist()
    item_cols = df.columns[item_cols]
    for col in item_cols:
        df[col] = df[col].apply(lambda x: x if pd.isna(x) or x.is_integer() else np.nan) # NaN non-integer values
        df[col] = df[col].astype('Int64') # convert to int

    thresh_cols = df.columns.str.contains('Threshold').tolist()
    thresh_cols = df.columns[thresh_cols]
    for col in thresh_cols:
        df[col] = df[col].apply(lambda x: x if pd.isna(x) or x.is_integer() else np.nan)
        df[col] = df[col].astype('Int64')

    total_cols = df.columns.str.contains('Total').tolist()
    total_cols = df.columns[total_cols]
    for col in total_cols:
        df[col] = df[col].apply(lambda x: x if pd.isna(x) or x.is_integer() else np.nan)
        df[col] = df[col].astype('Int64')

    # bool cols (all)
    bool_cols = df.select_dtypes(include = bool)
    for col in bool_cols:
        df[col] = df[col].astype('Int64')

    return df


2 Cleaning

In [ ]:
def Clean_Data(df):

    # clean non-questionnaire data
    df = preprocess_nonques_features(df)

    # clean questionnaire data
    df = preprocess_ques_features(df)

    return df

## 1.3 EDA Functions

3 EDA Plots

In [ ]:
def single_summary_plot(df, tic_freq):

    statistics = df.describe()

    plt.figure(figsize=(6, 4))
    for index, row in statistics.iterrows():
        plt.plot(statistics.columns, row, label=index)

    plt.xlabel('Column')
    plt.ylabel('Value')
    plt.title('Summary Statistics')
    plt.legend(loc = 'lower right')
    plt.xticks(statistics.columns[::tic_freq])
    plt.show()

In [ ]:
def summary_plot(dfs, dfs_names):

    # subplot dimensions
    num_dfs = len(dfs)
    num_rows = num_dfs // 2 + num_dfs % 2

    # plot
    plt.figure(figsize=(15, 5 * num_rows))
    for i, df in enumerate(dfs):

        statistics = df.describe()

        plt.subplot(num_rows, 2, i+1)
        for index, row in statistics.iterrows():
            plt.plot(statistics.columns, row, label=index)

        plt.xlabel('Column')
        plt.ylabel('Value')
        plt.title(f'{dfs_names[i]} Summary')
        plt.xticks(statistics.columns[::2])
        plt.legend(loc = 'center right')

    plt.tight_layout()
    plt.show()

## 1.4 Processing Functions

### 4.1 Duplicates

In [ ]:
def find_duplicates_features(df):
    dupe_cols = np.transpose(df).duplicated()
    dupe_cols = df.columns[dupe_cols].tolist()
    dupes_of = {}
    for col_name in dupe_cols:
        col_values = df[col_name]
        dupes = [other_col for other_col in df.columns if (other_col != col_name) and df[other_col].equals(col_values)]
        dupes_of[col_name] = dupes
        print(f'{col_name} is a duplicate of: {dupes}')

Final

In [ ]:
def drop_duplicate_features(df):
    dupe_cols = np.transpose(df).duplicated()
    dupe_cols = df.columns[dupe_cols].tolist()
    df = df.drop(columns = dupe_cols)
    return df

### 4.2 Outliers

In [ ]:
def column_contents(df):
    pot_cols = []
    for col in df.columns:
        if not df[col].isin([0, 1, pd.NA]).all():
            pot_cols.append(col)
    if len(pot_cols) == 0:
        print(f'Columns only contain 0, 1 and <NA>')
    else:
        print(f'Columns contining more than 0, 1 and <NA>: {pot_cols}')
    return pot_cols

In [ ]:
def mad(df):
    median = df.median()
    deviations = np.abs(df - median)
    mad_val = deviations.median() # MAD
    return mad_val

def modified_zscore(df, threshold = 3.5):
    median = df.median()
    mad_val = mad(df) # MAD
    modified_zscores = 0.6745 * (df - median) / mad_val # modified Z-score
    return np.abs(modified_zscores) > threshold

def find_outlier_cols(df, threshold = 3.5): # counts also

    outlier_df = modified_zscore(df)
    outlier_cols = set()
    for col in outlier_df.columns:
        outlier_list = []
        for i, val in outlier_df[col].items():
            if pd.notna(val) and val:
                outlier_list.append(df.iloc[i][col])
        if outlier_list:
            outlier_cols.add(col)
            print(f'{col} outlier count: {len(outlier_list)}')

    # return outlier_df
    outlier_cols = list(outlier_cols)
    outlier_df = df[outlier_cols]
    return outlier_df

In [ ]:
def plot_outlier_cols(outlier_df, title, legend=True):
    outlier_df.plot(figsize = (5, 3))
    plt.title(title)
    plt.xlabel('Index')
    plt.ylabel('Total Score')
    if legend:
        plt.legend(loc='upper right')
    else:
        plt.gca().legend().set_visible(False)
    plt.show()

def plot_datetime_features(df):
    plt.figure(figsize=(10, 4))
    for i, col in enumerate(datetime_cols):
        plt.subplot(1, 2, i + 1)
        df[col].hist(bins = 100, color = 'grey', edgecolor = 'white')
        plt.title(col)
        plt.xlabel('Date')
        plt.ylabel('Frequency')
        plt.grid(False)
    plt.tight_layout()
    plt.show()

def plot_outlier_cols2(outlier_df, title, color):

    plot_cols = 4
    plot_rows = len(outlier_df.columns)//4 + 1
    plt.figure(figsize = (plot_cols*3, plot_rows*3))

    for i, col in enumerate(outlier_df.columns, start = 1):
        plt.subplot(plot_rows, 4, i)
        outlier_df[col].plot(color=color)
        plt.title(col)
        plt.xlabel('Index')
        plt.ylabel('Total Score')

    plt.suptitle(title, y = 1, fontsize = 24)
    plt.tight_layout()
    plt.show()

Final

In [ ]:
def replace_ques_outliers(df): # shortens search using quartiles

    item_cols = [col for col in df.columns if 'Item' in col]
    item_df = df[item_cols]

    total_cols = [col for col in df.columns if 'Total' in col]
    total_df = df[total_cols]

    # replace negative totals with nan
    for col in total_df:
        df.loc[df[col] < 0, col] = np.nan # replace in df
        total_df.loc[total_df[col] < 0, col] = np.nan # replace in total_df

    # find col contents
    def create_pot_cols(df):
        pot_cols = []
        for col in df.columns:
            if not df[col].isin([0, 1, pd.NA]).all():
                pot_cols.append(col)
        return pot_cols

    # detect whether cols are just 0,1,na
    item_pot_cols = create_pot_cols(item_df)
    total_pot_cols = create_pot_cols(total_df)

    # potential columns containing outliers (shortening the search, only these can contain outliers)
    total_pot_df = total_df[total_pot_cols]
    item_pot_df = item_df[item_pot_cols]

    # outlier counter and df function
    def create_outlier_df(df, threshold = 3.5):

        outlier_df = modified_zscore(df)
        outlier_cols = set()
        for col in outlier_df.columns:
            outlier_list = []
            for i, val in outlier_df[col].items():
                if pd.notna(val) and val:
                    outlier_list.append(df.iloc[i][col])
            if outlier_list:
                outlier_cols.add(col)

        # return outlier_df
        outlier_cols = list(outlier_cols)
        outlier_df = df[outlier_cols]
        return outlier_df

    # find outliers in variables
    total_outlier_df = create_outlier_df(total_pot_df)
    item_outlier_df = create_outlier_df(item_pot_df)

    # replace outliers with nan
    for col in total_outlier_df:
        df.loc[df[col] > 200, col] = np.nan
        total_outlier_df.loc[total_outlier_df[col] > 200, col] = np.nan

    for col in item_outlier_df:
        df.loc[df[col] > 50, col] = np.nan
        item_outlier_df.loc[item_outlier_df[col] > 50, col] = np.nan

    df.loc[df['Total14'] > 37, 'Total14'] = np.nan
    df.loc[df['Item70'] > 3, 'Item70'] = np.nan
    df.loc[df['Item132'] > 5, 'Item132'] = np.nan

    return df


In [ ]:
def replace_outliers(df):
    df.loc[df['AgeAtReferralRequest'] == 0, 'AgeAtReferralRequest'] = np.nan
    df = replace_ques_outliers(df)
    return df

### 4.3 Constant and Quasi Constant Features

In [ ]:
def drop_const_features(df):
    const_cols = [col for col in df.columns if df[col].nunique() == 1]
    df = df.drop(columns = const_cols)
    return df

In [ ]:
def plot_datetime_features(df):
    plt.figure(figsize = (10,4))
    for i, col in enumerate(datetime_cols, 1):
        plt.subplot(1, len(datetime_cols), i)
        plt.hist(df[col], bins = 20, color = 'grey', edgecolor = 'white')
        plt.xlabel(col)
        plt.ylabel('Frequency')
        plt.title(col)
    plt.tight_layout()
    plt.show()

Final

In [ ]:
def quasi_percentage(df):

    # ordinal variables
    ordinal_df = df.select_dtypes(include = ['int64', 'Int64'])

    # select col modes
    modes = ordinal_df.mode() # mode of each col
    modes = modes.iloc[0] # select modes only

    # calc quasi percentage
    quasi_percentages = (ordinal_df == modes).sum() / len(ordinal_df) # how quasi a col is

    return quasi_percentages

# drop quasi features
def drop_quasi_features(df, threshold):

    # calculate quasi percentage
    quasi_percentages = quasi_percentage(df)

    # 0.995 is about 3 observations
    exceeding_threshold_columns = quasi_percentages[quasi_percentages > threshold].index
    df = df.drop(columns = exceeding_threshold_columns)
    return df

### 4.4 Correlation

In [ ]:
def select_future_features(df):
    EndDesc_cols = [col for col in df.columns if 'EndDesc' in col]
    #EndDescShort_cols = [col for col in df.columns if 'EndDescShort' in col] # none
    future_vars = df.drop(columns = ['ReliableChangeDesc', 'Recovery', 'ReliableRecovery'] + EndDesc_cols)
    return future_vars

In [ ]:
def generate_corr_matricies(df, load_matrices):

    file_path = '/content/drive/MyDrive/Data/Dissertation_Data/'

    if not load_matrices:
        # select explanatory variables, removing info on future
        future_df = select_future_features(df)

        # create correlation matrices
        kendall_corr = future_df.corr(method='kendall').abs()
        spearman_corr = future_df.corr(method='spearman').abs()
        kendall_corr.to_csv(file_path + 'kendall_corr.csv') # save
        spearman_corr.to_csv(file_path + 'spearman_corr.csv') # save

    else:
        kendall_corr = pd.read_csv(file_path + 'kendall_corr.csv', index_col=0)
        spearman_corr = pd.read_csv(file_path + 'spearman_corr.csv', index_col=0)

    return kendall_corr, spearman_corr

In [ ]:
def plot_correlation_matrices(corr_matricies):

    plt.figure(figsize=(8, 3))

    plt.subplot(121)
    sns.heatmap(corr_matricies[0], annot = False, cmap = 'flare', fmt = ".2f", xticklabels = False, yticklabels = False)
    plt.title('Kendall Correlation Matrix')

    plt.subplot(122)
    sns.heatmap(corr_matricies[1], annot = False, cmap = 'flare', fmt = ".2f", xticklabels = False, yticklabels = False)
    plt.title('Spearman Correlation Matrix')

    plt.tight_layout()
    plt.show()

In [ ]:
# count correlated variable pairs above each threshold
def count_correlated_pairs(correlation_matrix, thresholds):

    counts = {}
    for threshold in thresholds:
        correlated_vars = (correlation_matrix.abs() > threshold)
        # get upper triangle
        np.fill_diagonal(correlated_vars.values, False)
        upper_triangle = correlated_vars.values[np.triu_indices_from(correlated_vars, k=1)]
        count = np.sum(upper_triangle)
        counts[threshold] = count
    return counts

def correlated_features_above_treshold(matricies, thresholds):

    print('Kendall Correlation:')
    kendall_counts = count_correlated_pairs(matricies[0], thresholds)
    for threshold, count in kendall_counts.items():
        print(f'{threshold * 100}%: {count}')

    print('Spearman Correlation:')
    spearman_counts = count_correlated_pairs(matricies[1], thresholds)
    for threshold, count in spearman_counts.items():
        print(f'{threshold * 100}%: {count}')


In [ ]:
def drop_corr_cols(df, corr_matrix, threshold, ignore=True):

    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    corr_cols = [col for col in upper_tri.columns if any(upper_tri[col] > threshold)]

    try:
        df = df.drop(corr_cols, axis=1)
    except KeyError as e:
        if ignore:
            #print(f"Ignoring KeyError: {e}")
            pass
        else:
            raise

    return df

Final

In [ ]:
def remove_corr_features(df, load_matrices, threshold):

    kendall_corr, spearman_corr = generate_corr_matricies(df, load_matrices)

    # remove highly correlated variables
    df = drop_corr_cols(df, kendall_corr, threshold)
    df = drop_corr_cols(df, spearman_corr, threshold)

    return df


### 4.5 Missing Values within Columns

In [ ]:
def drop_missing_values_col(df, threshold):

    missing_value_percentage = df.isna().mean(axis=0) # cols

    drop_cols = missing_value_percentage[missing_value_percentage > threshold].index
    df = df.drop(drop_cols, axis=1)

    return df

### 4.6 Missing Values within Rows

In [ ]:
def drop_missing_values_row(df, threshold):

    missing_value_percentage = df.isna().mean(axis=1) # rows

    drop_cols = missing_value_percentage[missing_value_percentage > threshold].index
    df = df.drop(drop_cols, axis=0)

    return df

### 4.7 Scaling

In [ ]:
def standardise(df):
    scaler = StandardScaler()
    df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    return df

### 4.8 Imputation

In [ ]:
def impute_data(df, method, n_neighbours=None, max_iter=None):

    original_dtypes = df.dtypes.to_dict()

    #EndDescShort_cols = [col for col in df.columns if 'EndDescShort' in col] # none
    EndDesc_cols = [col for col in df.columns if 'EndDesc' in col]
    nontarget_df = df.drop(columns = ['ReliableChangeDesc', 'Recovery', 'ReliableRecovery'] + EndDesc_cols)

    datetime_cols = nontarget_df.select_dtypes(include=['datetime64[ns]']).columns
    for col in datetime_cols:
        nontarget_df[col] = pd.to_numeric(nontarget_df[col])

    if method == 'none':
        nontarget_mat = nontarget_df.values

    elif method == 'mean':
        imputer = SimpleImputer(strategy='mean')
        nontarget_mat = imputer.fit_transform(nontarget_df) # *numpy array*

    elif method == 'median':
        imputer = SimpleImputer(strategy='median')
        nontarget_mat = imputer.fit_transform(nontarget_df)

    elif method == 'knn':
        imputer = KNNImputer(n_neighbors=n_neighbours)
        nontarget_mat = imputer.fit_transform(nontarget_df)

    elif method == 'iterative':
        imputer = IterativeImputer(max_iter=max_iter, random_state=2001)
        nontarget_mat = imputer.fit_transform(nontarget_df)

    nontarget_df.iloc[:, :] = nontarget_mat.tolist()

    # add imputed data to df
    for col in nontarget_df.columns:
        df[col] = nontarget_df[col]

    if method == 'none':
        pass

    else:
        for col, dtype in original_dtypes.items():
            if dtype == 'datetime64[ns]':
                pass # keep as float
            else:
                try:
                    df[col] = df[col].astype(dtype)
                except (ValueError, TypeError):
                    # *convert to original dtype*
                    df[col] = df[col].round().astype(int)
                    df[col] = df[col].astype(dtype)

    return df


In [ ]:
# def multi_impute_data(df, method='knn', n_neighbours=3, iter=1):

#     EndDesc_cols = [col for col in df.columns if 'EndDesc' in col]
#     #EndDescShort_cols = [col for col in df.columns if 'EndDescShort' in col] # none
#     nontarget_df = df.drop(columns = ['ReliableChangeDesc', 'Recovery', 'ReliableRecovery'] + EndDesc_cols)

#     datetime_cols = nontarget_df.select_dtypes(include=['datetime64[ns]']).columns
#     for col in datetime_cols:
#         nontarget_df[col] = nontarget_df[col].astype(int) / 10**9

#     ordinal_cols = list(nontarget_df.select_dtypes(include=['Int64', 'int64']).columns)
#     imputed_dfs = []

#     for i in range(iter):

#         copy_df = nontarget_df.copy()

#         if method == 'mean':
#             mean_imputer = SimpleImputer(strategy='mean')
#             copy_df[nontarget_df.columns] = mean_imputer.fit_transform(copy_df[nontarget_df.columns])

#         elif method == 'median':
#             median_imputer = SimpleImputer(strategy='median')
#             copy_df[nontarget_df.columns] = median_imputer.fit_transform(copy_df[nontarget_df.columns])

#         elif method == 'knn':
#             KNN_imputer = KNNImputer(n_neighbors=n_neighbours)
#             copy_df[nontarget_df.columns] = KNN_imputer.fit_transform(copy_df)

#         imputed_dfs.append(copy_df)

#     df = np.mean(imputed_dfs, axis=0)
#     print(df.dtypes)
#     for col in datetime_cols:
#         df[col] = pd.to_datetime(df[col], unit='ns')

#     # convert imputed values for ordinal variables to discrete values
#     for col in ordinal_cols:
#         df[col] = df[col].round().astype(int)

#     return df


### 4.9 Feature Engineering

In [ ]:
def create_date_features(df):

    # df['ReferralYear'] = df['ReferralDate'].dt.year
    # df['ReferralMonth'] = df['ReferralDate'].dt.month
    # df['ReferralWeek'] = df['ReferralDate'].dt.isocalendar().week
    # df['ReferralDay'] = df['ReferralDate'].dt.day
    # df['ReferralHour'] = df['ReferralDate'].dt.hour
    # df['ReferralWeekDay'] = df['ReferralDate'].dt.dayofweek
    # df['ReferralYearDay'] = df['ReferralDate'].dt.dayofyear

    df['YearofQuestionnaire'] = df['DateOfQuestionnaire'].dt.year
    df['MonthofQuestionnaire'] = df['DateOfQuestionnaire'].dt.month
    df['WeekofQuestionnaire'] = df['DateOfQuestionnaire'].dt.isocalendar().week
    df['DayofQuestionnaire'] = df['DateOfQuestionnaire'].dt.day
    df['HourofQuestionnaire'] = df['DateOfQuestionnaire'].dt.hour
    df['WeekDayofQuestionnaire'] = df['DateOfQuestionnaire'].dt.dayofweek
    df['YearDayofQuestionnaire'] = df['DateOfQuestionnaire'].dt.dayofyear

    return df

In [ ]:
def feature_engineering(df):

    df = create_date_features(df)

    return df

### 4.10 Feature Importance

In [ ]:
def find_important_features(df, k_features, target='Recovery'):

    """
    k is the number of features each feature selection method should select.
    NOT the number of features returned

    """

    if k_features == None:
        important_features = df.columns

    else:
        features = df.dropna()
        EndDesc_cols = [col for col in df.columns if 'EndDesc' in col]
        explanatory = features.drop(['Recovery', 'ReliableRecovery', 'ReliableChangeDesc'] + EndDesc_cols, axis = 1)
        target = features[target]

        # fishers score
        kbest_fisher = SelectKBest(score_func=f_classif, k=k_features)
        selected_fisher = kbest_fisher.fit_transform(explanatory, target)
        index_fisher = kbest_fisher.get_support(indices=True)
        names_fisher = explanatory.columns[index_fisher]

        # mutual information gain
        def mutual_info_classif_wseed(X, y):
            return mutual_info_classif(X, y, random_state=11)
        kbest_gain = SelectKBest(score_func=mutual_info_classif_wseed, k=k_features)
        selected_gain = kbest_gain.fit_transform(explanatory, target)
        index_gain = kbest_gain.get_support(indices=True)
        names_gain = explanatory.columns[index_gain]

        important_combined = set(names_fisher).union(set(names_gain))
        important_features = list(important_combined)

    return important_features

In [ ]:
def select_important_features(df, k_features, target='Recovery'):

    important_features = find_important_features(df, k_features, target='Recovery')

    EndDesc_cols = [col for col in df.columns if 'EndDesc' in col]
    df = df[['Recovery', 'ReliableRecovery', 'ReliableChangeDesc'] + EndDesc_cols + important_features]

    return df

### 4 Prepare Data

In [ ]:
def Prepare_Data(df,
                 quasi_thresh=1.0,
                 corr_thresh=1.0,
                 load_matrices=True,
                 col_thresh=1.0,
                 row_thresh=1.0,
                 imputation_method='knn',
                 n_neighbours=10,
                 max_iter=10,
                 k_features=200):

    df = drop_duplicate_features(df)
    df = replace_outliers(df)
    df = drop_const_features(df)
    df = drop_quasi_features(df, quasi_thresh)
    df = remove_corr_features(df, corr_thresh, load_matrices)
    df = drop_missing_values_col(df, col_thresh)
    df = drop_missing_values_col(df, row_thresh)
    #df = standardise(df)
    df = impute_data(df, imputation_method, n_neighbours, max_iter)
    #df = feature_engineering(df)
    df = select_important_features(df, k_features)

    return df

## 1.5 Modelling

### Summary Plots

In [ ]:
def ModelSelection_Summary(model):

# pr curve

    scores = model[0]
    preds = model[1]
    actuals = model[2]

    print('Average accuracy score: {0}'.format(np.average(scores)))

    prec, recall, _ = metrics.precision_recall_curve(actuals, preds)
    print('AUPRC score: {0}\n'.format(metrics.auc(recall, prec)))

    # plot pr curve
    plt.figure(figsize=(12,4))
    plt.subplot(121)
    plt.plot(recall, prec, marker='.')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.show()

# confusion matrix

    # probabilities to binary
    threshold = 0.5
    binary_preds = [1 if pred >= threshold else 0 for pred in preds]

    # plot confusion matrix
    conf_matrix = confusion_matrix(actuals, binary_preds)

    plt.subplot(121)
    sns.heatmap(conf_matrix, annot = True, fmt = 'd', cmap = 'Purples')
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    plt.show()

### XGBoost

In [ ]:
def XGBoost_ModelSelection(df, target, selector, param_grid, k=5):

    # dataset
    sample = df.dropna(subset = [target])
    EndDesc_cols = [col for col in df.columns if 'EndDesc' in col] #EndDescShort_cols = [col for col in df.columns if 'EndDescShort' in col] # none
    X = sample.drop(['ReliableChangeDesc', 'ReliableRecovery', 'Recovery'] + EndDesc_cols, axis = 1)
    y = sample[target]
    cols = X.columns

    # machine learning algorithm
    classifier = XGBClassifier()

    # feature selection method
    if selector == 'SelectFromModel':
        selector = SelectFromModel(classifier)
    elif selector == 'RFE':
        selector = RFE(classifier)
    else:
        # fill this #
        raise ValueError('Unsupported selector type')

    # pipeline
    pipeline = Pipeline([("FS", selector), ("classifier", classifier)])

    # initialise lists
    scores, preds, actuals = [], [], []

    # cross validation and hyperparameter tuning
    outer_cv = StratifiedKFold(n_splits = k, shuffle = True)
    inner_cv = StratifiedKFold(n_splits = k, shuffle = True) # both set to k for now
    for train_index, test_index in outer_cv.split(X, y):

        # outer CV train and test sets
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # hyper-parameter tuning
        grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=inner_cv, scoring='accuracy', verbose=0, n_jobs=-1)
        grid_search.fit(X_train, y_train)
        print("Inner CV accuracy: {}".format(grid_search.best_score_)) # validation sets

        # optimal model
        estimator = grid_search.best_estimator_

        # count features selected
        support = estimator.named_steps['FS'].get_support()
        num_feat = np.sum(support)
        print("Number of selected features {0}".format(num_feat))

        # features selected
        col_index = np.where(support)[0]
        col_names = [cols[col] for col in col_index]
        print("Selected features {0}".format(col_names))

        # hyperparameters selected
        print("Max depth {0}".format(estimator.named_steps["classifier"].max_depth))
        print("Number of trees {0}".format(estimator.named_steps["classifier"].n_estimators))
        print("Learning rate {0}".format(estimator.named_steps["classifier"].learning_rate))
        print("Minimum child weight {0}".format(estimator.named_steps["classifier"].min_child_weight))
        print("Subsample {0}".format(estimator.named_steps["classifier"].subsample))
        print("Colsample bytree {0}".format(estimator.named_steps["classifier"].colsample_bytree))
        print("Gamma {0}".format(estimator.named_steps["classifier"].gamma))
        print("Lambda {0}".format(estimator.named_steps["classifier"].reg_lambda))
        print("Alpha {0}".format(estimator.named_steps["classifier"].reg_alpha))

        # evaluating optimised model on test
        predictions = estimator.predict(X_test)
        score = metrics.accuracy_score(y_test, predictions)
        scores.append(score)
        print('Outer CV accuracy: {}'.format(score)) # test sets

        print("--------------------------------------------------")

        probs = estimator.predict_proba(X_test)[:, 1]
        preds.extend(probs)
        actuals.extend(y_test)

    return scores, preds, actuals


## BERT

In [ ]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import torch

class BERTClassifier(BaseEstimator, TransformerMixin):
    def __init__(self, model_name='bert-base-uncased', max_length=128):
        self.model_name = model_name
        self.max_length = max_length
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertForSequenceClassification.from_pretrained(model_name)

    def fit(self, X, y):
        self.model.train()
        return self

    def predict(self, X):
        inputs = self.tokenizer(X.tolist(), return_tensors='pt', truncation=True, padding=True, max_length=self.max_length)
        with torch.no_grad():
            outputs = self.model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, axis=1)
        return preds.numpy()

    def predict_proba(self, X):
        inputs = self.tokenizer(X.tolist(), return_tensors='pt', truncation=True, padding=True, max_length=self.max_length)
        with torch.no_grad():
            outputs = self.model(**inputs)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=1)
        return probs.numpy()

def BERT_ModelSelection(df, text_column, target, param_grid, k=5):
    # dataset to model
    sample = df.dropna(subset=[target])  # remove missing target values
    X = sample[text_column]
    y = sample[target]

    # machine learning model
    classifier = BERTClassifier()

    # pipeline
    pipeline = Pipeline([('classifier', classifier)])

    # initialize lists
    scores = []
    preds = []
    actuals = []

    # k-fold CV
    kf = StratifiedKFold(n_splits=k, shuffle=True)
    for train_index, test_index in kf.split(X, y):
        # train and test data for CV
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # finding optimal models, hyperparameter tuning
        grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=kf, scoring='accuracy', verbose=0, n_jobs=-1)
        grid_search.fit(X_train, y_train)
        print("Internal CV Accuracy of estimator: {}".format(grid_search.best_score_))

        # best estimator
        estimator = grid_search.best_estimator_

        # print hyperparameters selected
        # For BERT, hyperparameters can include learning rate, batch size, etc.
        # Example:
        print("BERT Model used: {}".format(estimator.named_steps["classifier"].model_name))
        print("Max length used: {}".format(estimator.named_steps["classifier"].max_length))

        # predicting the test data with the optimized models
        predictions = estimator.predict(X_test)
        score = accuracy_score(y_test, predictions)
        scores.append(score)
        print('Accuracy performance on this test set: {}'.format(score))

        print("--------------------------------------------------")

        probs = estimator.predict_proba(X_test)[:, 1]
        preds.extend(probs)
        actuals.extend(y_test)

    return scores, preds, actuals

# A2) Cleaning

## 2.1 Overview

In [5]:
from google.colab import drive
import pandas as pd
import numpy as np

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# statistics
from scipy import stats

# scaling
from sklearn.preprocessing import MinMaxScaler # normalisation
from sklearn.preprocessing import Normalizer # works on rows not features
from sklearn.preprocessing import StandardScaler # standardisation

# missing values
import missingno as msno
#from scipy.stats import chi2_contingency

# imputation
from statsmodels.imputation.mice import MICEData
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

# machine learning models
from xgboost import XGBClassifier

# feature selection
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectFromModel

# fine-tuning
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

# evaluation
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix

In [6]:
drive.mount("/content/drive") # access google drive folder

file_path = '/content/drive/MyDrive/Data/Dissertation_Data/'
df = pd.read_csv(file_path + 'mental_health.csv', delimiter = ',') # csv
raw_df = df.copy()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
print('Sample of data:\n')
raw_df.head(3)

Sample of data:



,Unnamed: 0,IAPTus_Num,Referral Date,Age_ReferralRequest_ReceivedDate,EthnicDescGroupCode,EthnicCategoryGroupShortCode,GenderIdentity,SexualOrientationDesc,EndDesc,EndDescGroupShort,...,Item216,Item217,Item218,Item219,Item220,Item221,Item222,Item223,Item224,Item225
0,1,24475,08/09/2018,5.099020,1.0,1.0,2,NaN,Mutually agreed completion of treatment,Seen and treated,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,24476_1,10/04/2019,4.358899,1.0,1.0,2,NaN,Mutually agreed completion of treatment,Seen and treated,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,24476_2,28/03/2021,4.582576,1.0,1.0,2,NaN,Termition of treatment earlier than Care Profe...,Seen and treated,...,0,0,1,1,1,0,0,0,0,0


In [8]:
print('Data structure:')
print('\nNumber of rows:', raw_df.shape[0])
print('Number of columns:', raw_df.shape[1])
print('\nType:', type(raw_df))

Data structure:

Number of rows: 728
Number of columns: 279

Type: <class 'pandas.core.frame.DataFrame'>


In [9]:
# data types
print('Data types:\n')
raw_df.dtypes

Data types:



Unnamed: 0                            int64
IAPTus_Num                           object
Referral Date                        object
Age_ReferralRequest_ReceivedDate    float64
EthnicDescGroupCode                 float64
                                     ...   
Item221                              object
Item222                              object
Item223                              object
Item224                              object
Item225                              object
Length: 279, dtype: object

## 2.2 Non-Questionnaire Data Preprocessing



In [11]:
df = cleanf.rename_features(df)

In [ ]:
print('Column names and types:\n')
df.iloc[:, 0:20].dtypes

Including the ID variables, make changes to the numerical data.

In [ ]:
df = create_referral_count(df)
df = clean_client_id(df)
df = convert_features_to_datetime(df)
df = convert_float_features_to_int(df)

Explanation:
- Create a new column called ReferralCount counting the referrals (from clientID)
- Converted ClientID from object to numeric
- Converted date variables to datetime types
- Converted relevant types from floats to integers

Viewing the unique values the object variables contain.

In [ ]:
object_vars = df.iloc[:, 0:20].select_dtypes(include = ['object'])

print('Unique values:\n')
for var in object_vars:
    unique_vals = df[var].unique()
    print(var)
    print(unique_vals)
    print()

Creating appropriate maps to convert from object data to numerical whilst also one hot encoding too.

In [ ]:
df = map_features(df) # gender, treated, reliablechangedesc, recovery, reliablerecovery
df = one_hot_encode_features(df) # enddesc, enddescshort
df = convert_to_int_features(df) # sexualorientation, daystoassessment, daystotreatment, carecontacts

In [ ]:
print('New data types:\n')
print('Column names and types:')
df.iloc[:, 0:26].dtypes

In [ ]:
# data sample
df.iloc[:, 0:26].head(3)

## 2.3 Questionnaire Data Preprocessing

Inspect the questionnaire data.

In [ ]:
# questionnaire data
df.iloc[:, 27:].head()

In [ ]:
object_vars2 = df.iloc[:, 27:].select_dtypes(include = ['object'])

print('Unique values:\n')
lines_code = 5
for var in object_vars2.iloc[:, :lines_code]:
    unique_vals = df[var].unique()
    print(var)
    print(unique_vals)
    print()

Should change '.' to NaN and then convert these to integers.

In [ ]:
# preprocess questionnaire data
df = preprocess_ques_features(df)

# A3) Explanatory Data Analysis

## 3.1 Non-questionnaire Data

General plots of non-questionnaire data.

In [ ]:
item1_loc = df.columns.get_loc('Item1')
nonques_df = df.iloc[:, :item1_loc]
#desc_df = nonques_df[nonques_df.columns[nonques_df.nunique() >= 10]]

In [ ]:
# plot non-questionnaire data
plot_features(nonques_df)

In [ ]:
for col in nonques_df.columns: # convert datetime to numeric
    if nonques_df[col].dtype == 'datetime64[ns]':
        nonques_df[col] = pd.to_numeric(nonques_df[col], errors='coerce')

dfs = [nonques_df]
dfs_names = ['Non-questionnaire Data']
summary_plot(dfs, dfs_names)

## 3.2 Questionnaire Data

General plots of questionnaire data.

In [ ]:
item_cols = [col for col in df.columns if 'Item' in col]
item_df = df[item_cols]

thresh_cols = [col for col in df.columns if 'Threshold' in col]
thresh_df = df[thresh_cols]

total_cols = [col for col in df.columns if 'Total' in col]
total_df = df[total_cols]

# check accounts for all questionnaire data
if (nonques_df.shape[1] + item_df.shape[1] + thresh_df.shape[1] + total_df.shape[1]) == df.shape[1]:
    print('All features are being considered')
else:
    print('All features are not being considered')

In [ ]:
plt.figure(figsize=(12, 10))

plt.subplot(221)
sns.lineplot(data=item_df, legend=False)
plt.xlabel('Observations')
plt.ylabel('Values')
plt.title('Item Variables')
plt.xticks([])

plt.subplot(222)
sns.lineplot(data=thresh_df, legend=False)
plt.xlabel('Observations')
plt.ylabel('Values')
plt.title('Threshold Variables')
plt.xticks([])

plt.subplot(223)
sns.lineplot(data=total_df, legend=False)
plt.xlabel('Observations')
plt.ylabel('Values')
plt.title('Total Variables')
plt.xticks([])

plt.tight_layout()
plt.show()

** **Add random features maybe?** **

In [ ]:
# questionnaire data summary
dfs = [item_df, total_df, thresh_df]
dfs_names = ['Item Variables', 'Total Variables', 'Threshold Variables']
summary_plot(dfs, dfs_names)

Items:
- Quite a lot

Totals:
- At least one negative value in Total14
- Some high values in Total21 too

Thresholds:
- Everything potentially ok in these variables

## 3.3 Stratified by Target (or End of Treatment)

In [ ]:
EndDesc_cols = [col for col in df.columns if 'EndDesc' in col]
target_cols = ['Recovery', 'ReliableRecovery', 'ReliableChangeDesc'] + EndDesc_cols
targets_df = df[target_cols]

In [ ]:
plot_features(targets_df)

In [ ]:
cont_tb1 = pd.crosstab(df['Recovery'], df['ReliableRecovery'], dropna=False)
cont_tb2 = pd.crosstab(df['Recovery'], df['ReliableChangeDesc'], dropna=False)
cont_tb3 = pd.crosstab(df['ReliableRecovery'], df['ReliableChangeDesc'], dropna=False)

cont_tbs = [cont_tb1, cont_tb2, cont_tb3]

In [ ]:
def plot_cont_tables(cont_tbs):

    def plot_cont_table(cont_tb, cmap):
        sns.heatmap(cont_tb, annot=True, fmt='d', cmap=cmap)

    plt.figure(figsize=(8, 6))

    plt.subplot(221)
    plot_cont_table(cont_tbs[0], 'Blues')
    plt.subplot(222)
    plot_cont_table(cont_tbs[1], 'Reds')
    plt.subplot(223)
    plot_cont_table(cont_tbs[2], 'Greens')

    plt.tight_layout()
    plt.show()

In [ ]:
plot_cont_tables(cont_tbs)

In [ ]:
from scipy.stats import chi2_contingency

# Perform Chi-Square test
chi2, p, dof, expected = chi2_contingency(cont_tb1)

# Print the results
print(f"Chi-Square Statistic: {chi2}")
print(f"P-Value: {p}")
print(f"Degrees of Freedom: {dof}")
print("Expected Frequencies:")
print(expected)

# Interpret the p-value
alpha = 0.05
if p < alpha:
    print("There is a significant association between Recovery and ReliableRecovery.")
else:
    print("There is no significant association between Recovery and ReliableRecovery.")


In [ ]:
target = 'Recovery'

success_df = df[df[target] == 1]
notsuccess_df = df[df[target] == 0]

In [ ]:
target_corr1 = df.corr(method='kendall')[target]
target_corr2 = df.corr(method='spearman')[target]

In [ ]:
plt.figure(figsize=(10, 4))

plt.subplot(121)
sns.barplot(data=target_corr1, color='orange')
plt.axhline(y=0.3, color='r', linestyle='--')
plt.axhline(y=-0.3, color='r', linestyle='--')
plt.title(f'Correlation with {target}')
plt.xticks([])

plt.subplot(122)
sns.barplot(data=target_corr2, color='skyblue')
plt.axhline(y=0.3, color='r', linestyle='--')
plt.axhline(y=-0.3, color='r', linestyle='--')
plt.title(f'Correlation with {target}')
plt.xticks([])

plt.tight_layout()
plt.show()

In [ ]:
df.columns[abs(target_corr1) >= 0.3], df.columns[abs(target_corr2) >= 0.3]

# A4) Preparation

## 4.1 Duplicates

In [ ]:
dupe_rows = df.duplicated().sum()
dupe_cols = np.transpose(df).duplicated().sum()
print(f'Duplicate rows: {dupe_rows}')
print(f'Duplicate columns: {dupe_cols}')

In [ ]:
print('Duplicate details:\n')
find_duplicates_features(df)

Removing these duplicates from the dataset to reduce noise.

In [ ]:
df = drop_duplicate_features(df)

## 4.2 Outliers

**Non-questionnaire Outliers**

From EDA in section 3, these features seem to contain potential outliers.

In [ ]:
item1_start = df.columns.get_loc('Item1')
nonques_df = df.iloc[:, :item1_start]
observed_outliers = ['AgeAtReferralRequest', 'DaystoAssessment', 'DateOfQuestionnaire']
plot_features(nonques_df[observed_outliers])

In [ ]:
nonques_df[observed_outliers].min(), nonques_df[observed_outliers].max()

In [ ]:
age_outliers = df[df['AgeAtReferralRequest'] == 0]
print(f'Number of age outliers: {age_outliers.shape[0]}')
age_outliers.head(3)

In [ ]:
wait_outliers = df['DaystoAssessment'].sort_values(ascending=False)
wait_outliers.head(7)

In [ ]:
ques_date_outliers = df['DateOfQuestionnaire'].sort_values()
ques_date_outliers.head(3)

- Age of 0 seems to be outlier
- Waiting times to be assessed seem appropriate
- Questionnaire dates seems to be fair too

In [ ]:
# replace outliers with nan
df.loc[df['AgeAtReferralRequest'] == 0, 'AgeAtReferralRequest'] = np.nan

** **Use quartiles to find outliers** **

**Questionnaire Outliers**

In [ ]:
item_cols = [col for col in df.columns if 'Item' in col]
item_df = df[item_cols]

thresh_cols = [col for col in df.columns if 'Threshold' in col]
thresh_df = df[thresh_cols]

total_cols = [col for col in df.columns if 'Total' in col]
total_df = df[total_cols]

From the EDA, there exists negative total values. Removing these now will speed up the process.

In [ ]:
# replace negative totals with nan
for col in total_df:
    df.loc[df[col] < 0, col] = np.nan # replace in df
    total_df.loc[total_df[col] < 0, col] = np.nan # replace in total_df

Viewing potential features containing outliers.

In [ ]:
# detect whether cols are just 0,1,na
thresh_pot_cols = column_contents(thresh_df)
item_pot_cols = column_contents(item_df)
total_pot_cols = column_contents(total_df)

Initial Beliefs
- Treshold varibles contain no outliers as there are only 0 and 1s
- Constant columns and quasi constant columns will be explored and removed later
- Potential outliers in Items and Totals

In [ ]:
# potential columns containing outliers (shortening the search, only these can contain outliers)
total_pot_df = total_df[total_pot_cols]
item_pot_df = item_df[item_pot_cols]

# find outliers in Total variables
print('Outlier counts:\n')
total_outlier_df = find_outlier_cols(total_pot_df)
print('\n')
item_outlier_df = find_outlier_cols(item_pot_df)

In [ ]:
plot_outlier_cols(total_outlier_df, 'Total Variables')
plot_outlier_cols(item_outlier_df, 'Item Variables', legend=False)

Clearly the two massive distinct spikes are mistakes, removing these initally and takin another look before removing the other spike in the second plot seems resonable.

In [ ]:
# replace outliers with nan
for col in total_outlier_df:
    df.loc[df[col] > 200, col] = np.nan
    total_outlier_df.loc[total_outlier_df[col] > 200, col] = np.nan

for col in item_outlier_df:
    df.loc[df[col] > 50, col] = np.nan
    item_outlier_df.loc[item_outlier_df[col] > 50, col] = np.nan

In [ ]:
plot_outlier_cols(total_outlier_df, 'Total Variables', legend=False)
plot_outlier_cols(item_outlier_df, 'Item Variables', legend=False)

In [ ]:
total_outlier_df[total_outlier_df > 36].dropna(axis=1, how='all').dropna(axis=0, how='all')

Totals:
- Since there are so many instances of 36 being the highest total in Total21 it seems unlikely one achived higher, so removed.
- No large unusual points left.
- Everything else seems to be ok, since there are not many observations it would be hard to determine if one is an outlier if the difference from the rest is so small.

Items:
- This spike must be a mistake for Item70, it seems unlikely there was an option of 10 when the rest are 0 and 1.
- Similarly with the other reasoning, there are too little observations to accurately determine if any others are mistsakes since the difference in scoring is so small now.

In [ ]:
def plot_outlier_cols2(outlier_df, title, color):

    plot_cols = 4
    plot_rows = len(outlier_df.columns)//4 + 1
    plt.figure(figsize = (plot_cols*3, plot_rows*3))

    for i, col in enumerate(outlier_df.columns, start = 1):
        plt.subplot(plot_rows, 4, i)
        outlier_df[col].plot(color=color)
        plt.title(col)
        plt.xlabel('Index')
        plt.ylabel('Total Score')

    plt.suptitle(title, y = 1, fontsize = 24)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_outlier_cols2(total_outlier_df, 'Total Variables', 'limegreen')
plot_outlier_cols2(item_outlier_df, 'Item Variables', 'gold')

In [ ]:
df.loc[df['Total14'] > 37, 'Total14'] = np.nan
df.loc[df['Item70'] > 3, 'Item70'] = np.nan
df.loc[df['Item132'] > 5, 'Item132'] = np.nan

** **Alter item_df and total_df if analysis goes further** **

## 4.3 Constant and Quasi Constant Features (+99.5%)

Removing useless constant columns.

In [ ]:
df = drop_const_features(df)

Exploring the data for any quasi constant features.

In [ ]:
ordinal_cols = df.select_dtypes(include = ['Int64', 'int64']).columns
ordinal_df = df[ordinal_cols]

cont_cols = df.drop(columns = ordinal_cols).columns.tolist()
cont_df = df[cont_cols]

non_datetime_cols = df.select_dtypes(exclude = ['datetime64']).columns
non_datetime_df = df[non_datetime_cols]

datetime_cols = df.select_dtypes(include = ['datetime64']).columns
datetime_df = df[datetime_cols]

Viewing the variance across the continuous variables.

In [ ]:
# coefficient of variance
cov = df[non_datetime_cols].var() / df[non_datetime_cols].mean()

plt.figure(figsize = (10, 4))
plt.bar(cov.index, cov.values, color = 'dimgrey', edgecolor = 'white')
plt.xlabel('Variables')
plt.ylabel('Coefficient of Variation')
plt.title('Coefficient of Variation for each Variable')
plt.xticks([])
plt.show()

In [ ]:
print('Features with low CoV:\n')
cov.sort_values().head()

In [ ]:
quasi_percentages = quasi_percentage(df)
print('Quasi percentages:\n')
print(quasi_percentages.sort_values(ascending=False).head())

In [ ]:
#df[cov.sort_values().head().index].plot()
#df[quasi_percentages.sort_values(ascending=False).head().index].plot()

In [ ]:
plt.figure(figsize=(8, 4))
plt.hist(quasi_percentages.values, bins = 100, color = 'grey', edgecolor = 'white')
plt.xlabel('Percentage of Mode Occurrences')
plt.ylabel('Frequency')
plt.title('Distribution of Percentage of Mode Occurrences for Ordinal Variables')
plt.show()

In [ ]:
df = drop_quasi_features(df, threshold=0.995)

** **This is deleting a column that is the product of a categorical variable. Delete the whole rows too** **

## 4.4 Correlation (+99%, set to load)

** **Revisit, all variables can be used in Kendall and Spearman** **

In [ ]:
corr_matricies = generate_corr_matricies(df, load_matrices=True)
kendall_corr = corr_matricies[0]
spearman_corr = corr_matricies[1]

In [ ]:
corr_matricies = [kendall_corr, spearman_corr]
plot_correlation_matrices(corr_matricies)

In [ ]:
thresholds = [0.7, 0.8, 0.9, 0.95, 1.0]
correlated_features_above_treshold(corr_matricies, thresholds)

In [ ]:
df = remove_corr_features(df, load_matrices=True, threshold=0.99)

## 4.5 Missing Values within Features (+99%)

In [ ]:
print('Percentage missing data:')
print(df.isna().mean().mean() * 100)

In [ ]:
msno.matrix(df)

In [ ]:
df.isna().mean(axis=0).hist(color='teal', edgecolor='white', bins=df.shape[1]//10)
plt.axvline(x=0.5, color='firebrick', linestyle='--')  # Line at 50%
plt.title('Feature Missingness')
plt.ylabel('Frequency')
plt.xlabel('Missingness Percentage')

In [ ]:
df.isna().mean(axis=1).hist(color='salmon', edgecolor='white', bins=df.shape[0]//10)
plt.axvline(x=0.5, color='firebrick', linestyle='--')  # Line at 50%
plt.title('Observation Missingness')
plt.ylabel('Frequency')
plt.xlabel('Missingness Percentage')

In [ ]:
# for col1 in df.columns:
#     for col2 in df.columns:
#         # Only perform the chi-square test for pairs of categorical variables
#         if df[col1].dtype == 'int64'or'Int64' and df[col2].dtype == 'int64'or'Int64':
#             # Exclude performing the test for the same variable
#             if col1 != col2:
#                 # Create the contingency table
#                 ct = pd.crosstab(df[col1], df[col2])
#                 # Perform the chi-square test
#                 chi2, p, dof, exp = chi2_contingency(ct)
#                 if p <= 0.001:
#                 # Print the results
#                     print(f"Chi-square test for variables {col1} and {col2}:")
#                     #print(f"Chi-square statistic: {chi2}")
#                     print(f"P-value: {p}")
#                     #print(f"Degrees of freedom: {dof}")
#                     #print("Expected frequencies:")
#                     #print(exp)
#                     print()

Remove features with +50% missing data.

In [ ]:
df = drop_missing_values_col(df, threshold=0.9)

## 4.6 Missing Data within Observations (+99%)

In [ ]:
df.isna().sum(axis = 1)

In [ ]:
missing_value_percentage = df.isna().sum(axis = 1).sort_values() / df.shape[0]

plt.figure(figsize = (12, 4))

plt.subplot(121)
missing_value_percentage.plot(kind = 'bar', color='white', edgecolor='dimgrey')
plt.title('Proportion of Missing Values by Observation')
plt.xlabel('Observations')
plt.ylabel('Percentage of Missing Values')
plt.xticks([])
plt.axhline(y = 0.5, color='red', linestyle='--')  # line at 50%

plt.subplot(122)
plt.hist(missing_value_percentage, bins = 100, color='darkgrey', edgecolor='white')
plt.title('Proportion of Missing Values by Observation')
plt.xlabel('Percentage of Missing Values')
plt.ylabel('Frequency')
plt.axvline(x=0.5, color='red', linestyle='--')  # line at 50%

plt.tight_layout()
plt.show()

The aim is to retain as much data as possible, especially since there is not a lot as it is. Luckily non seem to exceed 70%, very few above 50%. It would be best not to remove anything at this point.

## 4.7 Scaling (none)

For classification tasks its recommended to standardise data.

In [ ]:
#df = standardise(df)

## 4.8 Imputation (knn, k = 7)

In [ ]:
df = impute_data(df, method='knn', n_neighbours=3)

** **Make sure not rounding to non-existing number** **
- could check that: before_df.unique() == after_df.unique()

## 4.9 Feature Engineering (none)

In [ ]:
temp_df = df.copy()
#df = temp_df.copy()

In [ ]:
df = feature_engineering(df)

In [ ]:
df.iloc[:,:-5].columns

## 4.10 Feature Importance

### Dataframes

In [ ]:
from sklearn.feature_selection import SelectKBest

from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import chi2 # neg values

In [ ]:
features = df.dropna()
target = 'Recovery'
EndDesc_cols = [col for col in df.columns if 'EndDesc' in col]
explanatory = features.drop(['Recovery', 'ReliableRecovery', 'ReliableChangeDesc'] + EndDesc_cols, axis = 1)
target = features[target]

### Univariate feature selection (used, k = 20)

In [ ]:
# fishers score
kbest_fisher = SelectKBest(score_func=f_classif, k=20)
selected_fisher = kbest_fisher.fit_transform(explanatory, target)
ind_fisher = kbest_fisher.get_support(indices=True)
names_fisher = explanatory.columns[ind_fisher]
print("Selected Features:")
print(names_fisher)

In [ ]:
# fishers score
f_values, p_values = f_classif(explanatory, target) # fisher score
feat_importances = pd.Series(f_values, index=explanatory.columns)
feat_importances = feat_importances[feat_importances > 10] # threshold
feat_importances.plot(kind='barh', color='darkblue')
plt.title("Univariate Feature Selection\nFisher's Score")

In [ ]:
# mutual information gain
def mutual_info_classif_wseed(X, y):
    return mutual_info_classif(X, y, random_state=11)
kbest_gain = SelectKBest(score_func=mutual_info_classif_wseed, k=20)
selected_gain = kbest_gain.fit_transform(explanatory, target)
ind_gain = kbest_gain.get_support(indices=True)
names_gain = explanatory.columns[ind_gain]
print("Selected Features:")
print(names_gain)

In [ ]:
# mutual information gain
importances = mutual_info_classif(explanatory, target, random_state=11) # mutual information gain
feat_importances = pd.Series(importances, index=explanatory.columns)
feat_importances = feat_importances[feat_importances > 0.05] # threshold
feat_importances.plot(kind='barh', color='darkgreen')
plt.title("Univariate Feature Selection\nMutual Information Gain")

In [ ]:
# Combine the two lists and get unique values using a set
important_combined = set(names_fisher).union(set(names_gain))

# Convert the set back to a list (optional) and print the unique feature names
important_features = list(important_combined)
print("Unique selected features:")
print(important_features)
print(f'Number of unique features: {len(important_features)}')

### Recursive feature elimination

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold

min_features_to_select = 1  # Minimum number of features to consider
clf = XGBClassifier(seed=2001)
cv = StratifiedKFold(5)

rfecv = RFECV(
    estimator=clf,
    step=1,
    cv=cv,
    scoring="accuracy",
    min_features_to_select=min_features_to_select,
    n_jobs=-1,
)
rfecv.fit(explanatory, target)

print(f"Optimal number of features: {rfecv.n_features_}")

In [ ]:
n_scores = len(rfecv.cv_results_["mean_test_score"])
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Mean test accuracy")
plt.errorbar(
    range(min_features_to_select, n_scores + min_features_to_select),
    rfecv.cv_results_["mean_test_score"],
    yerr=rfecv.cv_results_["std_test_score"],
)
plt.title("Recursive Feature Elimination\nwith Correlated Features")
plt.xlim(0, 50)
plt.show()

### Feature selection using SelectFromModel

Model-based and sequential feature selection with Ridge

In [ ]:
from sklearn.linear_model import RidgeCV

# Fit RidgeCV model
ridge = RidgeCV(alphas=np.logspace(-6, 6, num=5)).fit(explanatory, target)

# Calculate the importance of each feature
importance = np.abs(ridge.coef_)
feature_names = np.array(explanatory.columns)

# Filter features with importance greater than 0.0001
threshold = 0.00005
mask = importance > threshold
filtered_importance = importance[mask]
filtered_feature_names = feature_names[mask]

# Plot the filtered feature importances
plt.bar(height=filtered_importance, x=filtered_feature_names)
plt.title("Feature importances via coefficients (filtered)")
plt.xlabel("Features")
plt.ylabel("Importance")
plt.show()

Tree-based feature selection

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(explanatory, target)
clf.feature_importances_
model = SelectFromModel(clf, prefit=True)
X_new = model.transform(explanatory)
X_new.shape

### Sequential Feature Selection

### Correlation between target

In [ ]:
kendall_corr = df.corr(method='kendall')
spearman_corr = df.corr(method='spearman')

target = 'Recovery'
target_kendall_corr = (kendall_corr[target].abs() > 0.5).dropna()
target_spearman_corr = (spearman_corr[target].abs() > 0.5).dropna()

correlations = pd.DataFrame({'Kendall': target_kendall_corr, 'Spearman': target_spearman_corr})

plt.figure(figsize=(10, 6))
sns.heatmap(correlations, annot=True, cmap='coolwarm', fmt=".2f")
plt.title(f'Correlation of Features with {target}')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
correlations.plot(kind='bar', cmap='coolwarm')
plt.title(f'Correlation of Features with {target} (Absolute Values > 0.5)')
plt.xlabel('Features')
plt.ylabel('Correlation Coefficient')
plt.xticks(rotation=45)
plt.legend(title='Correlation Method')
plt.tight_layout()
plt.show()

# A5) Modelling

## 5.1 XGBoost

### Model 1

In [ ]:
param_grid1 = dict(
    #FS__threshold = ["mean", "min"],
    FS__n_features_to_select = [10, 20],
    classifier__n_estimators = [200, 500],
    classifier__max_depth = [3, 5, 10],
    classifier__learning_rate = [0.001, 0.01, 0.1],
    classifier__subsample = [0.5, 1.0],
    classifier__colsample_bytree = [0.5, 1.0])

In [ ]:
# model 1
xgb_spec_knn7_df = {
    'df': knn7_df,
    'target': 'ReliableRecovery',
    #'selector': 'SelectFromModel',
    'selector': 'RFE',
    'param_grid': param_grid1,
    'k': 5}

xgb_knn7_df = XGBoost_ModelSelection(**xgb_spec_knn7_df)

In [ ]:
ModelSelection_Summary(xgb_knn15)

In [ ]:
xgb_spec_knn10= {
    'df': knn10_df,
    'target': 'ReliableRecovery',
    'selector': 'SelectFromModel',
    'param_grid': param_grid1,
    'k': 5}

xgb_knn10 = XGBoost_ModelSelection(**xgb_spec_knn10)

In [ ]:
ModelSelection_Summary(xgb_knn10)

In [ ]:
xgb_spec_knn7= {
    'df': knn7_df,
    'target': 'ReliableRecovery',
    'selector': 'SelectFromModel',
    'param_grid': param_grid1,
    'k': 5}

xgb_knn7 = XGBoost_ModelSelection(**xgb_spec_knn7)

In [ ]:
ModelSelection_Summary(xgb_knn7)

### Model 2

In [ ]:
# hyperparameters for XGBoost and feature selector
param_grid2 = dict(
    FS__threshold = ["mean", "median"],
    classifier__n_estimators = [400, 500, 600],
    classifier__max_depth = [5, 7, 10],
    classifier__learning_rate = [0.05, 0.1, 0.5],
    classifier__min_child_weight = [0, 1],
    classifier__subsample = [0.8, 1],
    classifier__colsample_bytree = [0.8, 1],
    classifier__gamma = [0, 0.01],
    classifier__lambda = [0, 0.01],
    classifier__alpha = [0, 0.01])

### Final Model

In [ ]:
# Initialize XGBoost classifier with best hyperparameters
best_params = {
    'n_estimators': 200,
    'max_depth': 5,
    'learning_rate': 0.1,
    'min_child_weight': None,
    'subsample': 1.0,
    'colsample_bytree': 0.5,
    'gamma': None,
    'reg_lambda': None,
    'reg_alpha': None
}

xgb_classifier = XGBClassifier(**best_params)

# Feature selector
feature_selector = SelectFromModel(xgb_classifier)

# Create a pipeline
pipeline = Pipeline([
    ('FS', feature_selector),
    ('classifier', xgb_classifier)])

# Train the model on the entire dataset
pipeline.fit(X, y)

# B) Machine Learning

## Loading

### Libraries

In [ ]:
# preprocessing
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# eda
import seaborn as sns
from scipy import stats

# processing
from sklearn.preprocessing import MinMaxScaler # normalisation
from sklearn.preprocessing import Normalizer # works on rows not features
from sklearn.preprocessing import StandardScaler # standardisation

from statsmodels.imputation.mice import MICEData
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer # MICE (returning single rather than multiple)

from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import chi2 # neg values

# modelling
from xgboost import XGBClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix

### Data Processing

In [ ]:
# create different processed datasets
data, raw_df = load_data()
data = Clean_Data(data)

prep1_df = Prepare_Data(
    df=data,
    quasi_thresh=0.995,
    corr_thresh=0.95,
    load_matrices=True,
    col_thresh=0.5,
    row_thresh=0.5,
    imputation_method='iterative',
    #n_neighbours=3,
    max_iter=10,
    k_features=150)

## Fine-Tuning

In [ ]:
param_grid1 = dict(
    FS__threshold = ["mean", "median"],
    #FS__n_features_to_select = [10, 15],
    classifier__n_estimators = [200, 400, 500],
    classifier__max_depth = [3, 5],
    classifier__learning_rate = [0.01, 0.1, 0.5],
    classifier__subsample = [0.5, 0.8],
    classifier__colsample_bytree = [0.5, 0.8])

# model 1
xgb_spec1 = {
    'df': prep1_df,
    'target': 'ReliableRecovery',
    'selector': 'SelectFromModel',
    #'selector': 'RFE',
    'param_grid': param_grid1,
    'k': 10}

xgb_model1 = XGBoost_ModelSelection(**xgb_spec1)

In [ ]:
ModelSelection_Summary(xgb_model1)

In [ ]:
import pickle

file_path = '/content/drive/MyDrive/Data/Dissertation_Data/xgb_model1.pkl'

with open(file_path, 'wb') as file:
    pickle.dump(xgb_model1, file)


## Modelling

In [ ]:
def XGBoost_Train(df, target, selector, param_grid, k=5):

    # dataset
    sample = df.dropna(subset = [target])
    EndDesc_cols = [col for col in df.columns if 'EndDesc' in col] #EndDescShort_cols = [col for col in df.columns if 'EndDescShort' in col] # none
    X = sample.drop(['ReliableChangeDesc', 'ReliableRecovery', 'Recovery'] + EndDesc_cols, axis = 1)
    y = sample[target]
    cols = X.columns

    # machine learning algorithm
    classifier = XGBClassifier()

    # feature selection method
    if selector == 'SelectFromModel':
        selector = SelectFromModel(classifier)
    elif selector == 'RFE':
        selector = RFE(classifier)
    else:
        # fill this #
        raise ValueError('Unsupported selector type')

    # pipeline
    pipeline = Pipeline([("FS", selector), ("classifier", classifier)])

    # initialise lists
    scores, preds, actuals = [], [], []

    # cross validation and hyperparameter tuning
    outer_cv = StratifiedKFold(n_splits = k, shuffle = True)
    for train_index, test_index in outer_cv.split(X, y):

        # outer CV train and test sets
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # fit data
        pipeline.fit(X_train, y_train)

        # evaluating optimised model on test
        predictions = pipeline.predict(X_test)
        score = metrics.accuracy_score(y_test, predictions)
        scores.append(score)
        #print('Accuracy: {}'.format(score)) # test sets

        probs = pipeline.predict_proba(X_test)[:, 1]
        preds.extend(probs)
        actuals.extend(y_test)

    print('Overall accuracy: {}'.format(np.mean(scores))) # test sets

    print("--------------------------------------------------")

    return scores, preds, actuals


In [ ]:
def xgb_grid(params_str):
    params = params_str.split(', ')
    param_grid = {
        'FS__threshold': params[0],
        'classifier__max_depth': int(params[1]) if params[1] != 'None' else None,
        'classifier__n_estimators': int(params[2]) if params[2] != 'None' else None,
        'classifier__learning_rate': float(params[3]) if params[3] != 'None' else None,
        'classifier__min_child_weight': float(params[4]) if params[4] != 'None' else None,
        'classifier__subsample': float(params[5]) if params[5] != 'None' else None,
        'classifier__colsample_bytree': float(params[6]) if params[6] != 'None' else None,
        'classifier__gamma': float(params[7]) if params[7] != 'None' else None,
        'classifier__reg_lambda': float(params[8]) if params[8] != 'None' else None,
        'classifier__reg_alpha': float(params[9]) if params[9] != 'None' else None
    }
    return param_grid

In [ ]:
param_grid1 = xgb_grid('"mean", 3, 200, 0.1, None, 0.5, 0.8, None, None, None')
param_grid2 = xgb_grid('"mean", 5, 200, 0.5, None, 0.5, 0.5, None, None, None')
param_grid3 = xgb_grid('"median", 5, 500, 0.01, None, 0.5, 0.5, None, None, None')
param_grid4 = xgb_grid('"median", 5, 500, 0.01, None, 0.8, 0.5, None, None, None')
param_grid5 = xgb_grid('"median", 5, 200, 0.5, None, 0.8, 0.5, None, None, None')
param_grid6 = xgb_grid('"mean", 5, 400, 0.01, None, 0.8, 0.8, None, None, None')
param_grid7 = xgb_grid('"mean", 5, 400, 0.1, None, 0.8, 0.8, None, None, None')
param_grid8 = xgb_grid('"median", 5, 400, 0.01, None, 0.8, 0.8, None, None, None')
param_grid9 = xgb_grid('"median", 5, 200, 0.5, None, 0.8, 0.8, None, None, None')
param_grid10 = xgb_grid('"mean", 5, 200, 0.1, None, 0.8, 0.8, None, None, None')

param_grids = [param_grid1, param_grid2, param_grid3, param_grid4, param_grid5,
               param_grid6, param_grid7, param_grid8, param_grid9, param_grid10]

In [ ]:
for i in range(10):
    xgb_spec = {
        'df': prep1_df,
        'target': 'ReliableRecovery',
        'selector': 'SelectFromModel',
        #'selector': 'RFE',
        'param_grid': param_grids[i],
        'k': 10}

    XGBoost_Train(**xgb_spec)

## Modelling 2

# C) Large Language Models

## Loading

### Libraries

In [ ]:
# preprocessing
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# eda
import seaborn as sns
from scipy import stats

# processing
from sklearn.preprocessing import MinMaxScaler # normalisation
from sklearn.preprocessing import Normalizer # works on rows not features
from sklearn.preprocessing import StandardScaler # standardisation

from statsmodels.imputation.mice import MICEData
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer # MICE (returning single rather than multiple)

from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import chi2 # neg values

# modelling
from xgboost import XGBClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix

### Data Processing

In [ ]:
# create different processed datasets
data, raw_df = load_data()
data = Clean_Data(data)

prep1_df = Prepare_Data(
    df=data,
    quasi_thresh=0.999,
    corr_thresh=0.999,
    load_matrices=True,
    col_thresh=0.9,
    row_thresh=0.9,
    imputation_method='iterative',
    #n_neighbours=3,
    max_iter=10,
    k_features=200)

## Modelling 1

In [ ]:
!pip install datasets;
!pip install evaluate;
!pip install -U accelerate;
!pip install -U transformers;

import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import GPT2Tokenizer
from transformers import GPT2ForSequenceClassification
import evaluate
from transformers import TrainingArguments, Trainer

In [ ]:
def BERT_Train(df, target='Recovery', k=5):

    # dataset
    EndDesc_cols = [col for col in df.columns if 'EndDesc' in col]
    explanatory_df = df.drop(['ReliableChangeDesc', 'ReliableRecovery', 'Recovery'] + EndDesc_cols, axis = 1)
    text = explanatory_df.apply(lambda row: row.to_json(), axis=1)
    text_df = pd.DataFrame({
        'text': text,
        'label': df[target]})
    text_df = text_df.dropna()
    dataset = Dataset.from_pandas(text_df)

    # tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokenizer.pad_token = tokenizer.eos_token
    def tokenize_function(examples):
        return tokenizer(examples['text'], padding='max_length', truncation=True)
    tokenized_datasets = dataset.map(tokenize_function, batched=True)

    # model
    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

    # evaluation
    metric = evaluate.load("accuracy")
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)

    # arguments
    training_args = TrainingArguments(
        output_dir="test_trainer",
        #evaluation_strategy="epoch",
        per_device_train_batch_size=1,  # Reduce batch size here
        per_device_eval_batch_size=1,   # Optionally, reduce for evaluation as well
        gradient_accumulation_steps=4)

    # cross validation
    cv = StratifiedKFold(n_splits=k, shuffle=True)
    splits = cv.split(text_df['text'], text_df['label'])
    for fold, (train_index, test_index) in enumerate(splits):

        # train and test sets
        train_split = tokenized_datasets.select(train_index.tolist())
        test_split = tokenized_datasets.select(test_index.tolist())

        # trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_set,
            eval_dataset=test_set,
            compute_metrics=compute_metrics,)

        trainer.train()

        trainer.evaluate()

    print("--------------------------------------------------")


In [ ]:
BERT_Train(prep1_df, target='Recovery', k=5)

## Modelling

In [ ]:
!pip install datasets;
!pip install evaluate;
!pip install -U accelerate;
!pip install -U transformers;

In [ ]:
EndDesc_cols = [col for col in prep1_df.columns if 'EndDesc' in col]
explanatory_df = prep1_df.drop(['ReliableChangeDesc', 'ReliableRecovery', 'Recovery'] + EndDesc_cols, axis = 1)

text = explanatory_df.apply(lambda row: row.to_json(), axis=1)
text_df = pd.DataFrame({
    'text': text,
    'label': prep1_df['Recovery']})

text_df = text_df.dropna()

#!pip install datasets;

from datasets import Dataset, DatasetDict
import pandas as pd

dataset = Dataset.from_pandas(text_df)

train_size = int((text_df.shape[0]) * 0.8)
train_dataset = dataset.select(range(train_size))
test_dataset = dataset.select(range(train_size, len(dataset)))

dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset})

In [ ]:
### Tokenizer ###

from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# small train and evaluation sets

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(79Z))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(79))

### Initialise base model ###

from transformers import GPT2ForSequenceClassification
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)

### Evaluate method ###

#!pip install evaluate;
import evaluate
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   return metric.compute(predictions=predictions, references=labels)

### Fine-tune (Trainer method) ###

#!pip install -U accelerate;
#!pip install -U transformers;
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
   output_dir="test_trainer",
   #evaluation_strategy="epoch",
   per_device_train_batch_size=1,  # Reduce batch size here
   per_device_eval_batch_size=1,   # Optionally, reduce for evaluation as well
   gradient_accumulation_steps=4)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=small_train_dataset,
   eval_dataset=small_eval_dataset,
   compute_metrics=compute_metrics,)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
# trainer.save_model("test_trainer")
# tokenizer.save_pretrained("test_trainer")

In [ ]:
# tokenize data
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# small train and evaluation sets
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(50))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(50))

from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# !pip install evaluate
import evaluate
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# !pip install -U accelerate transformers
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="test_trainer",
    per_device_train_batch_size=1,  # Reduce batch size here
    per_device_eval_batch_size=1,   # Optionally, reduce for evaluation as well
    gradient_accumulation_steps=4)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
pip install optuna

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
import numpy as np
import optuna
import evaluate

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Create small training and evaluation datasets
small_train_dataset = tokenized_datasets["train"].shuffle(seed=2001).select(range(50))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=2001).select(range(50))

# Load model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Metric for evaluation
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Function to initialize the model
def model_init():
    return BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Training arguments template
training_args = TrainingArguments(
    output_dir="test_trainer",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch"
)

# Trainer
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)

# Define hyperparameter search space
def hyperparameter_search_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-4, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [1, 2, 4])
    }

# Conduct hyperparameter search
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    n_trials=10,
    compute_objective=lambda metrics: metrics["eval_accuracy"],
    hp_space=hyperparameter_search_space
)

print("Best Hyperparameters:\n", best_trial.hyperparameters)


## Old

** **Consider whether order of features matters - its sequential** **

In [ ]:
bert_param_grid = {
    'classifier__max_length': [128, 256, 512]
    # Add more hyperparameters if needed
}

bert1_model = BERT_ModelSelection(serialised_df, 'serialised_data', 'Recovery', bert_param_grid, k=5)

In [ ]:
ModelSelection_Summary(bert1_model)